In [14]:
class Parser:
    def __init__(self, input_file):
        self.file = open(input_file, 'r') # opening the file
        self.commands = self.file.readlines() # reading the file
        self.current_command = None # currentcommand decleration
        self.current_index = 0 # cureent index set to 0

    def hasMoreCommands(self):
        return self.current_index < len(self.commands) # checking if there are more commands to translate

    def advance(self):
        while self.hasMoreCommands(): # advance when there are more commands translate
            line = self.commands[self.current_index].strip() # fetching that index line
            self.current_index += 1 # increasing the index value
            if line and not line.startswith("//"): # checking if it is a comment or not
                self.current_command = line.split("//")[0].strip() # stroing that command in the current command
                break

    def commandType(self):
        if self.current_command.startswith("push"): # checking for push
            return "C_PUSH"
        elif self.current_command.startswith("pop"): # checking for pop
            return "C_POP"
        else: # checking for arithmetic
            return "C_ARITHMETIC"

    def arg1(self):  # for push and pop type commands taking out the first argument that is type of value it stores like temp, const etc
        if self.commandType() == "C_ARITHMETIC":# ignoring if arithmetic
            return self.current_command
        return self.current_command.split()[1]

    def arg2(self):
        if self.commandType() in ["C_PUSH", "C_POP"]:
            return int(self.current_command.split()[2]) # getting the value given as second argument in push and pop command

In [23]:
class CodeWriter:
    def __init__(self, output_file):
        self.output = open(output_file, 'w')  # Opens the file in write mode
        self.label_counter = 0  # Counter to generate unique labels

    def writeArithmetic(self, command): # for writing codes for arithmetic operations
        if command == "add": # for add
            self._write_binary_op("M=M+D")
        elif command == "sub": # for sub
            self._write_binary_op("M=M-D")
        elif command == "neg": # for negative
            self._write_unary_op("M=-M")
        elif command == "eq": # for equality
            self._write_compare_op("JEQ")
        elif command == "gt": # for checking greater than
            self._write_compare_op("JGT")
        elif command == "lt": # for checking less than
            self._write_compare_op("JLT")
        elif command == "and": # and operation
            self._write_binary_op("M=M&D")
        elif command == "or": # or operation
            self._write_binary_op("M=M|D")
        elif command == "not": # not command
            self._write_unary_op("M=!M")

    def writePushPop(self, command, segment, index): # passing the segment that is arg1 and index that is arg2 with the command that is either push or pop
        if command == "C_PUSH": # checking if it is a push command
            if segment == "constant":  # if it is a constant
                self._write_code(f"@{index}\nD=A")  # writing the corresponding code to the file
                self._push_d_to_stack()  # pushing the integer to the top of the stack
            elif segment in ["local", "argument", "this", "that"]: # if segment is local or argunment or this or that
                segment_pointer = self._get_segment_pointer(segment)  # first we get a pointer that is the place to which it is pointed by the argument in the RAM
                self._write_code(f"@{index}\nD=A\n@{segment_pointer}\nA=M+D\nD=M") # writing the corresponding assembly code in the assembly file
                self._push_d_to_stack() # pushing the data value to the stack
            elif segment == "temp":  # if it is temp variable
                self._write_code(f"@{5+int(index)}\nD=M")  # 5 is the base address for temp registers so we add the index to 5 to get the original location
                self._push_d_to_stack() # pushing the data value to the stack
            elif segment == "pointer":  # if it is a pointer type
                pointer_base = "THIS" if index == "0" else "THAT"  # if the value is 0 then this else that
                self._write_code(f"@{pointer_base}\nD=M") # writing to the output file for the base pointer
                self._push_d_to_stack() # pushing the data value to the stack
            elif segment == "static": # for static variable
                self._write_code(f"@Static.{index}\nD=M") # printing static first and then the index of the static variable
                self._push_d_to_stack() # pushing the data value to the stack

        elif command == "C_POP":  # pop command
            if segment in ["local", "argument", "this", "that"]: # if it is of local, argumrnt, this or that type
                segment_pointer = self._get_segment_pointer(segment) # getting the segmetn related to it
                self._write_code(f"@{index}\nD=A\n@{segment_pointer}\nD=M+D\n@R13\nM=D") # writing hte code
                self._pop_stack_to_d() # popping from the stack
                self._write_code("@R13\nA=M\nM=D") # stroign the variable in memory
            elif segment == "temp": # for temp variable
                self._pop_stack_to_d() # popping from stack
                self._write_code(f"@{5+int(index)}\nM=D") # writing to the output file
            elif segment == "pointer": # for pointer based pop
                pointer_base = "THIS" if index == "0" else "THAT" # getting the base of pointer
                self._pop_stack_to_d() # popping the value from the stack
                self._write_code(f"@{pointer_base}\nM=D") # writing the code
            elif segment == "static": # ofr static
                self._pop_stack_to_d() # popping from stack
                self._write_code(f"@Static.{index}\nM=D") # writing the code

    def close(self):
        self.output.close() # closing the file

    # Helper functions

    def _write_code(self, code):
        self.output.write(code + "\n") # writing the output in the file

    def _push_d_to_stack(self): # assembly code for pushing into the stack
        self._write_code("@SP\nA=M\nM=D\n@SP\nM=M+1")

    def _pop_stack_to_d(self): # popping from the stack
        self._write_code("@SP\nAM=M-1\nD=M")

    def _write_binary_op(self, operation): # writing binary code
        self._write_code("@SP\nAM=M-1\nD=M\nA=A-1\n" + operation) # writing the deafult code with the required code

    def _write_unary_op(self, operation):  # writing unary code
        self._write_code("@SP\nA=M-1\n" + operation) # writing the deafult code with the required code

    def _write_compare_op(self, jump_command): # writing the comparison operator
        self.label_counter += 1 # label counter
        true_label = f"TRUE_{self.label_counter}" # creating label names for true labels
        end_label = f"END_{self.label_counter}" # creating label names for false labels
        self._write_code(
            "@SP\nAM=M-1\nD=M\nA=A-1\nD=M-D\n"
            f"@{true_label}\nD;{jump_command}\n"
            "@SP\nA=M-1\nM=0\n"
            f"@{end_label}\n0;JMP\n"
            f"({true_label})\n@SP\nA=M-1\nM=-1\n"
            f"({end_label})"
        ) # command for comparison operators

    def _get_segment_pointer(self, segment):  # defining the segments
        segment_pointers = {
            "local": "LCL",
            "argument": "ARG",
            "this": "THIS",
            "that": "THAT"
        }
        return segment_pointers[segment] # returning segment pointers


def main(input_file, output_file):
    parser = Parser(input_file)  # declaring the parser for the input file
    code_writer = CodeWriter(output_file) # declaring the codewriter for the output file

    while parser.hasMoreCommands(): # cheking until parser can have more commands
        parser.advance() # advancing via the parser
        command_type = parser.commandType() # checkign the command type of the line

        if command_type == "C_ARITHMETIC":
            code_writer.writeArithmetic(parser.arg1()) # for arithmetic commands
        elif command_type in ["C_PUSH", "C_POP"]:
            code_writer.writePushPop(command_type, parser.arg1(), parser.arg2()) # fo rpush and pop operations

    code_writer.close() # closing the file


if __name__ == "__main__":
    input_vm_file = "/content/StaticTest.vm"  # VM file path
    output_asm_file = "/content/output_statictest.asm"  # desired assembly file path and name
    main(input_vm_file, output_asm_file)